In [ ]:
#Install neccessary libraries, especially DDLFS
pip install --upgrade  xgboost lightgbm pandas dask[complete] dask-ml[complete] distributed pyarrow joblib adlfs dask_lightgbm

In [ ]:
# Authenicating to Azure ML
ws = Workspace.from_config()
ws

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Dataset, Environment

In [9]:
keyvault=ws.get_default_keyvault()

In [2]:
import dask
import glob
import joblib
import fsspec

import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime
from dask.distributed import Client
from dask_lightgbm import LGBMRegressor
from dask_ml.xgboost import XGBRegressor
from azureml.core import Workspace, Dataset, Model

In [18]:
c = Client('10.0.0.5:8786')
# c = Client()
#you can also use the standalone version of dask with Client()

c

Client Scheduler: tcp://10.0.0.5:8786 Dashboard: http://10.0.0.5:8787/status,Cluster Workers: 5 Cores: 40 Memory: 295.49 GB


In [19]:
STORAGE_OPTIONS={'account_name': 'adlsdatalakegen6', 'account_key': keyvault.get_secret("adlsdatalakegen6")}


In [20]:
#file list
file_list = dd.read_csv("abfs://mltraining/ISDWeatherDelta/_symlink_format_manifest/year=*/month=*/manifest",storage_options=STORAGE_OPTIONS,engine='python', header =None )

In [21]:
file_list_pd = file_list.compute()
parquet_file_list = [file[0].replace("abfss", "abfs").replace("@adlsdatalakegen6.dfs.core.windows.net", "") for file in file_list_pd.values]

In [22]:
len(parquet_file_list)

37

In [24]:
# df.head()

import dask.dataframe as dd

STORAGE_OPTIONS={'account_name': 'adlsdatalakegen6', 'account_key': keyvault.get_secret("adlsdatalakegen6")}
df = dd.read_parquet(parquet_file_list, storage_options=STORAGE_OPTIONS,engine='pyarrow').repartition(npartitions=200).persist()
%time df.head()


CPU times: user 250 ms, sys: 35.6 ms, total: 286 ms
Wall time: 5.07 s


,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,...,presentWeatherIndicator,pastWeatherIndicator,precipTime,precipDepth,snowDepth,stationName,countryOrRegion,p_k,day,version
0,999999,53878,2008-04-20 03:40:00,35.419,-82.557,641.0,NaN,NaN,9.5,NaN,...,NaN,NaN,NaN,NaN,NaN,ASHEVILLE 13 S,US,999999-53878,20,1.0
1,999999,53877,2008-04-06 08:20:00,35.495,-82.614,656.0,NaN,NaN,7.2,NaN,...,NaN,NaN,NaN,NaN,NaN,ASHEVILLE 8 SSW,US,999999-53877,6,1.0
2,999999,53877,2008-04-13 12:35:00,35.495,-82.614,656.0,NaN,NaN,6.5,NaN,...,NaN,NaN,NaN,NaN,NaN,ASHEVILLE 8 SSW,US,999999-53877,13,1.0
3,999999,53877,2008-04-07 22:55:00,35.495,-82.614,656.0,NaN,NaN,16.2,NaN,...,NaN,NaN,NaN,NaN,NaN,ASHEVILLE 8 SSW,US,999999-53877,7,1.0
4,999999,53877,2008-04-30 09:00:00,35.495,-82.614,656.0,NaN,0.1,-2.1,NaN,...,NaN,NaN,1.0,0.0,NaN,ASHEVILLE 8 SSW,US,999999-53877,30,1.0


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR - _GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR - _GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
ERROR - _GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [15]:

%time df.shape[0].compute()

CPU times: user 5.96 s, sys: 792 ms, total: 6.75 s
Wall time: 2min 17s


73696631

In [ ]:
# pd_df = df.compute()

In [17]:
df = df.set_index(dd.to_datetime(df.datetime).dt.floor('d'), sorted=False).persist()
%time len(df)



CPU times: user 432 ms, sys: 13.4 ms, total: 445 ms
Wall time: 56.9 s


81837618

In [18]:
df.head()

,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,...,presentWeatherIndicator,pastWeatherIndicator,precipTime,precipDepth,snowDepth,stationName,countryOrRegion,p_k,day,version


In [19]:
df.tail()

,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,...,presentWeatherIndicator,pastWeatherIndicator,precipTime,precipDepth,snowDepth,stationName,countryOrRegion,p_k,day,version
2008-12-31,703670,25322,2008-12-31 01:56:00,58.424,-137.700,9.0,0.0,0.0,-9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,GUSTAVUS,US,703670-25322,31,1.0
2008-12-31,746940,03708,2008-12-31 21:00:00,35.460,-77.951,41.0,330.0,8.2,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,GOLDSBORO-WAYNE MUNI ARPT,US,746940-03708,31,1.0
2008-12-31,994420,99999,2008-12-31 07:00:00,30.250,-88.083,9.0,240.0,2.6,15.9,1018.7,...,NaN,NaN,NaN,NaN,NaN,DAUPHIN ISLAND AL,US,994420-99999,31,1.0
2008-12-31,703870,25338,2008-12-31 04:36:00,56.484,-132.360,13.0,80.0,8.2,-4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,WRANGELL AIRPORT,US,703870-25338,31,1.0
2008-12-31,564620,99999,2008-12-31 15:00:00,29.000,101.500,2994.0,160.0,2.0,3.1,NaN,...,NaN,NaN,NaN,NaN,NaN,JIULONG,CH,564620-99999,31,1.0


In [20]:
%time df.describe().compute()

CPU times: user 8.35 s, sys: 51.5 ms, total: 8.4 s
Wall time: 15.6 s


,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,presentWeatherIndicator,pastWeatherIndicator,precipTime,precipDepth,snowDepth,day,version
count,8.183762e+07,8.183762e+07,8.183762e+07,6.653191e+07,6.892623e+07,7.976075e+07,3.019107e+07,7.631862e+06,3.358177e+06,1.587158e+07,1.587158e+07,235416.000000,8.183762e+07,81837618.0
mean,3.804371e+01,-4.089479e+01,3.490733e+02,1.640719e+02,3.491411e+00,1.315999e+01,1.014005e+03,2.739949e+01,4.212506e+00,5.613270e+00,4.215797e+01,25.789037,1.580390e+01,1.0
std,2.078054e+01,7.845431e+01,5.336248e+02,1.150190e+02,2.893313e+00,1.153178e+01,9.067073e+00,2.785420e+01,3.032504e+00,8.274215e+00,5.651420e+02,71.053035,8.835149e+00,0.0
min,-9.000000e+01,-1.799830e+02,-3.500000e+02,0.000000e+00,0.000000e+00,-8.300000e+01,9.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000e+00,1.0
25%,3.423300e+01,-9.588300e+01,3.100000e+01,9.000000e+01,2.100000e+00,1.000000e+01,1.012000e+03,1.000000e+01,2.000000e+00,1.000000e+00,0.000000e+00,3.000000,8.000000e+00,1.0
50%,4.170100e+01,-7.688300e+01,1.680000e+02,2.300000e+02,4.100000e+00,1.700000e+01,1.016800e+03,2.100000e+01,6.000000e+00,1.000000e+00,0.000000e+00,16.000000,1.600000e+01,1.0
75%,4.866700e+01,1.421700e+01,4.020000e+02,3.100000e+02,6.700000e+00,2.700000e+01,1.031200e+03,6.300000e+01,8.000000e+00,6.000000e+00,5.000000e+00,67.500000,2.400000e+01,1.0
max,8.670000e+01,1.797500e+02,9.999000e+03,3.600000e+02,9.000000e+01,5.800000e+01,1.088800e+03,9.900000e+01,9.000000e+00,9.900000e+01,9.999000e+03,999.000000,3.100000e+01,1.0


In [ ]:
%time means = df.groupby(df.index).mean().compute()
means.head()

In [ ]:
## insert any Pandas-like Dask transformation code 
df['temperature'] = df['temperature']*(9/5)+32 # 'Merica

In [ ]:
# begin data prep for ML 
df = df.fillna(0)

In [ ]:
cols = list(df.columns)
cols = [col for col in cols if df.dtypes[col] != 'object' and col not in ['version', 'datetime']]
cols

In [ ]:
X = df[[col for col in cols if col not in ['temperature']]].persist()
y = df.temperature.persist()
# end data prep for ML

In [ ]:
xgb = XGBRegressor(n_estimators=16)
%time xgb.fit(X, y)

In [ ]:
%time y_pred = xgb.predict(X).compute()

In [ ]:
%time rmse = (((y.to_dask_array().compute()-y_pred)**2).mean())**.5
print(f'Training RMSE: {round(rmse, 3)}')

In [ ]:
lgbm = LGBMRegressor(n_estimators=16)
%time lgbm.fit(X, y)

In [ ]:
%time y_pred = lgbm.predict(X).compute()

In [ ]:
%time rmse2 = (((y.to_dask_array().compute()-y_pred)**2).mean())**.5
print(f'Training RMSE: {round(rmse2, 3)}')

In [ ]:
if rmse2 < rmse:
    best_model = 'lgbm'
else:
    best_model = 'xgb'

best_model

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR - _GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [ ]:
model_path = f'{best_model}_noaa_isd.joblib.dat'
joblib.dump(xgb, model_path) if best_model is 'xgb' else joblib.dump(lgbm, model_path)
model2 = joblib.load(model_path)

In [ ]:
model = Model.register(ws, 
                       model_name      = f'{best_model}-noaa-isd', 
                       model_path      = model_path,
                       description     = f'NOAA ISD temperature predictor')